---
title: Algorhythmic Experience Blog
description: This blog details some of my experiences at the Algorythmic Event as well as some code blocks to illustrate what each group showed. 
toc: true
layout: post
type: hacks
comments: true
---

# Actual Code for the Sorts

## Bubble Sort

In [2]:
public class BubbleSort {
    public static void bubbleSort(int[] data) {
        int length = data.length;

        for (int iIndex = 0; iIndex < length; iIndex++) {
            boolean swapped = false;

            for (int jIndex = 0; jIndex < length - iIndex - 1; jIndex++) {

                if (data[jIndex] > data[jIndex + 1]) {
                    int temp = data[jIndex];
                    data[jIndex] = data[jIndex + 1];
                    data[jIndex + 1] = temp;
                    swapped = true;
                }
            }

            if (!swapped) {
                break;
            }
        }
    }

    public static void main(String[] args) {
        int[] data = {5, 1, 4, 2, 8};
        System.out.println("Original array: ");
        for (int value : data) {
            System.out.print(value + " ");
        }
        System.out.println();

        bubbleSort(data);

        System.out.println("Sorted array: ");
        for (int value : data) {
            System.out.print(value + " ");
        }
    }
}

BubbleSort.main(null);


Original array: 
5 1 4 2 8 
Sorted array: 
1 2 4 5 8 

### Explanation

The bubble sorting algorithm is an algorithm that compares two consecutive elements of a list. If the two elements (example numbers) are in the right order, nothing needs to be changed. If the numbers are not in the right order, however, they will then be swapped so that the smaller numbers can be "bubbled" to the top of the list.

Bubble sort gets the job done of sorting the list; however, it does have an average time complexity of O(n^2), meaning that the number of operations performed will get larger and larger as the list in question gets larger and larger. Thus, even though bubble sort is one of the more simpler algorithms to understand, it may not be the best option when it comes to sorting.



## Quick Sort

In [5]:
public class QuickSort {
    public static void quickSort(int[] data, int left, int right) {
        if (left < right) {
            int pivotIndex = partition(data, left, right);
            quickSort(data, left, pivotIndex - 1);
            quickSort(data, pivotIndex + 1, right);
        }
    }

    private static int partition(int[] data, int left, int right) {
        int pivot = data[left];
        int leftIndex = left + 1;
        int rightIndex = right;

        while (true) {
            while (leftIndex <= rightIndex && data[leftIndex] <= pivot) {
                leftIndex++;
            }
            while (rightIndex >= leftIndex && data[rightIndex] >= pivot) {
                rightIndex--;
            }
            if (rightIndex <= leftIndex) {
                break;
            }
            int temp = data[leftIndex];
            data[leftIndex] = data[rightIndex];
            data[rightIndex] = temp;
        }

        data[left] = data[rightIndex];
        data[rightIndex] = pivot;

        return rightIndex;
    }

    public static void main(String[] args) {
        int[] data = {10, 7, 8, 9, 1, 5};
        System.out.println("Original array:");
        for (int value : data) {
            System.out.print(value + " ");
        }
        System.out.println();

        quickSort(data, 0, data.length - 1);

        System.out.println("Sorted array:");
        for (int value : data) {
            System.out.print(value + " ");
        }
    }
}

QuickSort.main(null);

Original array:
10 7 8 9 1 5 
Sorted array:
1 5 7 8 9 10 

### Explanation

Quick sort selects one element of a list as a pivot. The list is then rearranged so that all elements less than the pivot come before it, while all elements greater than the pivot come after it. This process guarantees that the pivot element is in its correct position in the sorted list. Quick sort then recursively applies the same logic to the list of elements before the pivot and the list of elements after the pivot until the entire list is sorted.

Quick sort has an average time complexity of O(n log n), similar to merge sort. However, its worst-case performance is O(n^2), which occurs when the list is already nearly sorted or when the smallest or largest element is always chosen as the pivot. 

## Merge Sort

In [6]:
public class MergeSort {
    public static int[] mergeSort(int[] data) {
        if (data.length < 2) {
            return data;
        }

        int middle = data.length / 2;
        int[] left = new int[middle];
        int[] right = new int[data.length - middle];

        for (int i = 0; i < middle; i++) {
            left[i] = data[i];
        }
        for (int i = middle; i < data.length; i++) {
            right[i - middle] = data[i];
        }

        mergeSort(left);
        mergeSort(right);

        return merge(left, right, data);
    }

    private static int[] merge(int[] left, int[] right, int[] data) {
        int leftIndex = 0, rightIndex = 0, mergeIndex = 0;

        while (leftIndex < left.length && rightIndex < right.length) {
            if (left[leftIndex] <= right[rightIndex]) {
                data[mergeIndex++] = left[leftIndex++];
            } else {
                data[mergeIndex++] = right[rightIndex++];
            }
        }

        while (leftIndex < left.length) {
            data[mergeIndex++] = left[leftIndex++];
        }

        while (rightIndex < right.length) {
            data[mergeIndex++] = right[rightIndex++];
        }

        return data;
    }

    public static void main(String[] args) {
        int[] data = {12, 11, 13, 5, 6, 7};
        System.out.println("Original array:");
        for (int value : data) {
            System.out.print(value + " ");
        }
        System.out.println();

        mergeSort(data);

        System.out.println("Sorted array:");
        for (int value : data) {
            System.out.print(value + " ");
        }
    }
}

MergeSort.main(null);

Original array:
12 11 13 5 6 7 
Sorted array:
5 6 7 11 12 13 

### Explanation

With the merge sorting algorithm, the algorithm takes one array and splits it in half. For each half of the array, the algorithm rearranges the array so that the elements are in the desired order. Once that has been done successfully, the algorithm merges the two arrays/lists back together so that the desired order is achieved for the entire array/list.

Merge sort, just like bubble sort, is also another way of sorting a list back into the desired order. However, merge sort is better than bubble sort in terms of time complexity, with merge sort having an average time complexity of O(nlogn). This means that the amount of operations performed will not be as large as the input size increases for a merge sort algorithm as it would be for a bubble sorting algorithm.

## Insertion Sort

In [4]:
public class InsertionSort {
    public static void insertionSort(int[] data) {
        for (int scanIndex = 1; scanIndex < data.length; scanIndex++) {
            int tmp = data[scanIndex];
            int minIndex = scanIndex;

            while (minIndex > 0 && tmp < data[minIndex - 1]) {
                data[minIndex] = data[minIndex - 1];
                minIndex--;
            }

            data[minIndex] = tmp;
        }
    }

    public static void main(String[] args) {
        int[] data = {9, 14, 3, 2, 43, 11, 58, 22};
        System.out.println("Original array:");
        for (int value : data) {
            System.out.print(value + " ");
        }
        System.out.println();

        insertionSort(data);

        System.out.println("Sorted array:");
        for (int value : data) {
            System.out.print(value + " ");
        }
    }
}

InsertionSort.main(null);

Original array:
9 14 3 2 43 11 58 22 
Sorted array:
2 3 9 11 14 22 43 58 

Insertion sort builds the final sorted list one item at a time. It removes one element per iteration and finds the location it belongs within the sorted part of the list, inserting it there. This process repeats until no unsorted elements remain.

Even though insertion sort has an average and worst-case time complexity of O(n^2), making it less efficient for large lists, it has several advantages. It is adaptive, meaning that its efficiency increases if the list is already partially sorted, and it is stable, keeping identical elements in the same order as they appear in the input. 

## Selection Sort

In [3]:
public class SelectionSort {
    public static void selectionSort(int[] data) {
        for (int scanIndex = 0; scanIndex < data.length; scanIndex++) {
            int minIndex = scanIndex;

            for (int compIndex = scanIndex + 1; compIndex < data.length; compIndex++) {
                if (data[compIndex] < data[minIndex]) {
                    minIndex = compIndex;
                }
            }

            if (minIndex != scanIndex) {
                int temp = data[scanIndex];
                data[scanIndex] = data[minIndex];
                data[minIndex] = temp;
            }
        }
    }

    public static void main(String[] args) {
        int[] data = {64, 25, 12, 22, 11};
        System.out.println("Original array:");
        for (int value : data) {
            System.out.print(value + " ");
        }
        System.out.println();

        selectionSort(data);

        System.out.println("Sorted array:");
        for (int value : data) {
            System.out.print(value + " ");
        }
    }
}

SelectionSort.main(null);

Original array:
64 25 12 22 11 
Sorted array:
11 12 22 25 64 

Selection sort divides the list into two parts: the sorted part at the start and the unsorted part at the rest of the list. Initially, the sorted part is empty, and the unsorted part contains all the elements. With each iteration, the algorithm selects the minimum (or maximum, depending on sorting order) element from the unsorted part and moves it to the end of the sorted part. This process is repeated until the unsorted part becomes empty, and the list is sorted.

Selection sort has a time complexity of O(n^2), making it inefficient for large lists. The primary advantage of selection sort lies in its simplicity and the fact that it makes the minimum possible number of swaps: only one swap for each element in the list, which could be beneficial in scenarios where writing data is significantly more expensive than reading data. 

# Collectables

In [8]:
import java.util.ArrayList;
import java.util.Collections;

class Node<T extends Comparable<T>> {
    T data;
    Node<T> next;

    Node(T data) {
        this.data = data;
        this.next = null;
    }
}

class CustomLinkedList<T extends Comparable<T>> {
    Node<T> head;

    public void add(T data) {
        if (head == null) {
            head = new Node<>(data);
        } else {
            Node<T> current = head;
            while (current.next != null) {
                current = current.next;
            }
            current.next = new Node<>(data);
        }
    }

    public void sort() {
        if (head == null || head.next == null) {
            return;
        }
        
        ArrayList<T> list = new ArrayList<>();
        Node<T> current = head;
        while (current != null) {
            list.add(current.data);
            current = current.next;
        }
        
        Collections.sort(list);
        
        current = head;
        for (T item : list) {
            current.data = item;
            current = current.next;
        }
    }

    @Override
    public String toString() {
        StringBuilder json = new StringBuilder("[");
        Node<T> current = head;
        while (current != null) {
            json.append(current.data.toString());
            if (current.next != null) {
                json.append(", ");
            }
            current = current.next;
        }
        json.append("]");
        return json.toString();
    }
}

class Person implements Comparable<Person> {
    String name;
    int age;

    Person(String name, int age) {
        this.name = name;
        this.age = age;
    }

    @Override
    public int compareTo(Person other) {
        if (this.age == other.age) {
            return this.name.compareTo(other.name);
        }
        return Integer.compare(this.age, other.age);
    }

    @Override
    public String toString() {
        return String.format("{\"name\": \"%s\", \"age\": %d}", name, age);
    }
}

// Demonstrate usage
public class Main {
    public static void main(String[] args) {
        CustomLinkedList<Person> people = new CustomLinkedList<>();
        people.add(new Person("Emaad", 17));
        people.add(new Person("Gumball", 12));
        people.add(new Person("Spencer", 30));
        people.add(new Person("Troll", 20));
        
        System.out.println("Original list: " + people);
        
        people.sort();
        
        System.out.println("Sorted list: " + people);
    }
}


Main.main(null);


Original list: [{"name": "Emaad", "age": 17}, {"name": "Gumball", "age": 12}, {"name": "Spencer", "age": 30}, {"name": "Troll", "age": 20}]
Sorted list: [{"name": "Gumball", "age": 12}, {"name": "Emaad", "age": 17}, {"name": "Troll", "age": 20}, {"name": "Spencer", "age": 30}]


# My Experience at the Algorythmic Event

Today (April 3), my group along with four other groups got to perform each of our sorting algorithms in front of the judges as well as people who happened to be interested in watching us perform. I enjoyed watching every single performance, as I was amazed at all of the groups were able to get creative with showing off their sorting algorithms in their own unique way. For example, the insertion sort algorithm performance was pretty cool in that it used rap lyrics as well as a beach theme in order to convey the mechanisms of insertion sort. Also, the bubble sort performance was funny to me and many others who attended the event since the theme was centered around a dating show and a person wanting to figure out who their best match is. 

In terms of our own performance, I thought that everyone (including me) did an amazing job. I feel like all of us were punctual whenever we were practicing, whether it was at lunch, during class, or outside of school. Ultimately, all of those practice sessions played of in the end, as we were able to perform on the stage without any issues and earned two 10/10s and 3 9/10s from the judges, which is quite amazing. We ended up in 3rd place for total scores, which we all agreed was a pretty solid ranking. 

Here are some captures of our practice sessions!

![]({{site.baseurl}}/images/IMG_0721.jpg)
![]({{site.baseurl}}/images/IMG_0722.jpg)
![]({{site.baseurl}}/images/IMG_0724.jpg)


Captures from the event 

![]({{site.baseurl}}/images/dating.png)
![]({{site.baseurl}}/images/dating2.png)

Dating show ^ (ft. Rachit aka Rachelina)


